<center><b>Model to Classify Intergers as Positive or Negative </b></center>

In [462]:
# Import required modules
import torch
from torch import nn

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

# Use device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"


In [463]:

# Set the hyperparameters fro data creation
NUM_CLASSES = 2
NUM_FEATURES = 4
RANDOM_SEED = 42

# 1. Create multi-class data
inputs =   np.array([[0,0,0,0], # 0
                     [0,0,0,1], # 1
                     [0,0,1,0], # 2
                     [0,0,1,1], # 3
                     [0,1,0,0], # 4
                     [0,1,0,1], # 5
                     [0,1,1,0], # 6
                     [0,1,1,1], # 7
                     [1,0,0,0], # 8
                     [1,0,0,1]])  # 9

outputs = np.array([[0,0], # 0
                   [0,1], # 1
                   [1,0], # 2
                   [0,1], # 3
                   [1,0], # 4
                   [0,1], # 5
                   [1,0], # 6
                   [0,1], # 7
                   [1,0], # 8
                   [0,1]]) # 9


# 2. Turn data into tensors
X = torch.from_numpy(inputs).type(torch.float)
y = torch.from_numpy(outputs).type(torch.FloatTensor)

# 3. Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=RANDOM_SEED)


In [464]:
# 4. Build model
class ModelEO(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
        """Initialize all required hyperparameters for a multi-class classification model.

        Args:
            input_features (int): Number of input features to the model.
            out_features (int): Number of output features of the model
                (how many classes there are).
            hidden_units (int): Number of hidden units between layers, default 8.
        """
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=output_features), # how many classes are there?
        )

    def forward(self, x):
        return self.linear_layer_stack(x)

In [465]:
# 5. Create an instance of BlobMOdel and send it to the target device
model_eo = ModelEO(input_features=NUM_FEATURES,
                    output_features=NUM_CLASSES,
                    hidden_units=8).to(device)

model_eo

ModelEO(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=2, bias=True)
  )
)

In [466]:
# 6. Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_eo.parameters(), lr=0.1)

In [467]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(torch.softmax(y_true, dim=1).argmax(dim=1), y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [468]:
# 7. Perform a single forward pass on the data (we'll need to put it to the target device for it to work)
model_eo(X_train.to(device))[:5]

tensor([[0.1287, 0.0967],
        [0.0655, 0.0102],
        [0.1297, 0.1336],
        [0.1085, 0.0773],
        [0.1433, 0.1145]], grad_fn=<SliceBackward0>)

In [469]:
model_eo(X_train.to(device))[0].shape, NUM_CLASSES

(torch.Size([2]), 2)

In [470]:
# 8. Make prediction logits with model
y_logits = model_eo(X_test.to(device))

# 9. Perform softmax calculation on logits across dimension 1 to get prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1)
print(y_logits[:5])
print(y_pred_probs[:5])

tensor([[0.1013, 0.0637],
        [0.1059, 0.1061],
        [0.0935, 0.1281],
        [0.1459, 0.1289],
        [0.1261, 0.1161]], grad_fn=<SliceBackward0>)
tensor([[0.5094, 0.4906],
        [0.4999, 0.5001],
        [0.4914, 0.5086],
        [0.5042, 0.4958],
        [0.5025, 0.4975]], grad_fn=<SliceBackward0>)


In [471]:
# 10. Which class does the model think is *most* likely at the index 0 sample?
print(y_pred_probs[1])
print(torch.argmax(y_pred_probs[1]))

tensor([0.4999, 0.5001], grad_fn=<SelectBackward0>)
tensor(1)


In [472]:
# 11. Fit the model
torch.manual_seed(42)

# Set number of epochs
epochs = 200

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

for epoch in range(epochs):
    ### Training
    model_eo.train()

    # 1. Forward pass
    y_logits = model_eo(X_train) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    #print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train)
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_eo.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = model_eo(X_test)
        test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
        # 2. Calculate test loss and accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        # print(y_pred)
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69625, Acc: 40.00% | Test Loss: 0.55141, Test Acc: 60.00%
Epoch: 10 | Loss: 0.66588, Acc: 60.00% | Test Loss: 0.57682, Test Acc: 40.00%
Epoch: 20 | Loss: 0.64063, Acc: 60.00% | Test Loss: 0.59097, Test Acc: 40.00%
Epoch: 30 | Loss: 0.60351, Acc: 80.00% | Test Loss: 0.59361, Test Acc: 40.00%
Epoch: 40 | Loss: 0.53508, Acc: 100.00% | Test Loss: 0.57856, Test Acc: 40.00%
Epoch: 50 | Loss: 0.43009, Acc: 100.00% | Test Loss: 0.54787, Test Acc: 40.00%
Epoch: 60 | Loss: 0.31050, Acc: 100.00% | Test Loss: 0.49931, Test Acc: 80.00%
Epoch: 70 | Loss: 0.21587, Acc: 100.00% | Test Loss: 0.44513, Test Acc: 80.00%
Epoch: 80 | Loss: 0.15264, Acc: 100.00% | Test Loss: 0.40322, Test Acc: 80.00%
Epoch: 90 | Loss: 0.11057, Acc: 100.00% | Test Loss: 0.36944, Test Acc: 80.00%
Epoch: 100 | Loss: 0.08189, Acc: 100.00% | Test Loss: 0.33862, Test Acc: 80.00%
Epoch: 110 | Loss: 0.06219, Acc: 100.00% | Test Loss: 0.31243, Test Acc: 80.00%
Epoch: 120 | Loss: 0.04847, Acc: 100.00% | Test Loss: 0

In [473]:
# 12. Save the model
from pathlib import Path

# 1.Models directory
MODEL_PATH = Path("models")

# 2. Create model save path
MODEL_NAME = "even_odd_classifier_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Save the model state dict
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model_eo.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

Saving model to: models/even_odd_classifier_model_0.pth


In [474]:
# Load and use model
# Create a new instance of our model class
loaded_model_eo = ModelEO(input_features=NUM_FEATURES,
                          output_features=NUM_CLASSES,
                          hidden_units=8).to(device)

# Load saved model state_dict
loaded_model_eo.load_state_dict(torch.load(f=MODEL_SAVE_PATH), strict=False)

loaded_model_eo

ModelEO(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=8, bias=True)
    (3): ReLU()
    (4): Linear(in_features=8, out_features=2, bias=True)
  )
)

In [475]:
# Use it for inference
loaded_model_eo.eval()

with torch.inference_mode():
    # 1. Forward pass
    y_logits = loaded_model_eo(X_test)
    y_preds = torch.softmax(y_logits, dim=1).argmax(dim=1)

    # 2. Calculate accuracy
    acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

# Print the result
print(f"Accuracy: {acc}% ")
X_test, y_preds

Accuracy: 100.0% 


(tensor([[1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 1., 0., 1.],
         [0., 0., 0., 0.],
         [0., 1., 1., 1.]]),
 tensor([0, 1, 1, 0, 1]))

In [476]:
# Test on data greater than 10
inputs = np.arange(100,300,1)

inputs

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
       126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
       139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
       152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
       165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
       178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
       191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
       204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
       217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229,
       230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242,
       243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255,
       256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268,
       269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 28

In [477]:
# Prepare the data
# We only need the last digit to know if a number is even or odd
last_digits = inputs % 10

# The binary equivalents of decimals 0 to 9
binary_representations =   np.array([[0,0,0,0], # 0
                                     [0,0,0,1], # 1
                                     [0,0,1,0], # 2
                                     [0,0,1,1], # 3
                                     [0,1,0,0], # 4
                                     [0,1,0,1], # 5
                                     [0,1,1,0], # 6
                                     [0,1,1,1], # 7
                                     [1,0,0,0], # 8
                                     [1,0,0,1]])  # 9

# Reshape the last_digits array to have the same number of dimensions as binary_representations
reshaped_last_digits = last_digits[:, np.newaxis]

X = np.empty((0, 4), int)
# Get the binary equivalents of the last_digits
for x in range(len(reshaped_last_digits)):
    X = np.append(X, binary_representations[reshaped_last_digits[x]], axis=0)

# Turn the data into tensors
X_inf = torch.from_numpy(X).type(torch.float)

X_inf[:10]  # Display the first 10 rows of X

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 1., 0., 1.],
        [0., 1., 1., 0.],
        [0., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 1.]])

In [478]:

# Peform the predictions and print the result
loaded_model_eo.eval()

with torch.inference_mode():
    # Forward pass
    logits = loaded_model_eo(X_inf)
    predictions = torch.softmax(logits, dim=1).argmax(dim=1)

# Print inputs against the predictions for the first 10
for x in range(20):
    print(f"Input: {inputs[x]} - Prediction: {predictions[x].item()}")


Input: 100 - Prediction: 0
Input: 101 - Prediction: 1
Input: 102 - Prediction: 0
Input: 103 - Prediction: 1
Input: 104 - Prediction: 0
Input: 105 - Prediction: 1
Input: 106 - Prediction: 0
Input: 107 - Prediction: 1
Input: 108 - Prediction: 0
Input: 109 - Prediction: 1
Input: 110 - Prediction: 0
Input: 111 - Prediction: 1
Input: 112 - Prediction: 0
Input: 113 - Prediction: 1
Input: 114 - Prediction: 0
Input: 115 - Prediction: 1
Input: 116 - Prediction: 0
Input: 117 - Prediction: 1
Input: 118 - Prediction: 0
Input: 119 - Prediction: 1
